In [ ]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
# ensure the below library is installed on your fabric environment
from geopy.geocoders import Nominatim

In [ ]:
df = spark.read.table('location')

In [ ]:
# Create geolocator 
def reverse_geocode(lat, lon):
    try:
        geolocator = Nominatim(user_agent="geoapi")
        location = geolocator.reverse((lat, lon), language="en", timeout=10)
        # Return whichever field is available (city, town, village, etc.)
        if location and "address" in location.raw:
            address = location.raw["address"]
            city = address.get("city")
            state = address.get("state")
            return f"{city}, {state}"
    except:
        return None

In [ ]:
# Register as UDF
reverse_geocode_udf = udf(reverse_geocode, StringType())

In [ ]:
# adding country_code and city attributes
df_with_location = df.\
                    withColumn("city", \
                    reverse_geocode_udf(col("latitude"), col("longitude")))

In [ ]:
# appending the data to the gold table
df_with_location.write.option("mergeSchema", "true").mode('overwrite').saveAsTable('location')